# 用增强学习玩车杆游戏

玩一局

In [1]:
import gym
env = gym.make('CartPole-v0') # 获得游戏环境
observation = env.reset() # 复位游戏环境,新一局游戏开始
print ('新一局游戏 初始观测 = {}'.format(observation))
for t in range(200):
    env.render()
    action = env.action_space.sample() # 随机选择动作
    print ('{}: 动作 = {}'.format(t, action))
    observation, reward, done, info = env.step(action) # 执行行为
    print ('{}: 观测 = {}, 本步得分 = {}, 结束指示 = {}, 其他信息 = {}'.format(
            t, observation, reward, done, info))
    if done:
        break
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
新一局游戏 初始观测 = [ 0.00974547  0.04064659  0.0486582  -0.04658112]
0: 动作 = 0
0: 观测 = [ 0.0105584  -0.15513809  0.04772658  0.26104819], 本步得分 = 1.0, 结束指示 = False, 其他信息 = {}
1: 动作 = 1
1: 观测 = [ 0.00745564  0.03927122  0.05294754 -0.01620744], 本步得分 = 1.0, 结束指示 = False, 其他信息 = {}
2: 动作 = 1
2: 观测 = [ 0.00824106  0.23359543  0.05262339 -0.29172597], 本步得分 = 1.0, 结束指示 = False, 其他信息 = {}
3: 动作 = 0
3: 观测 = [0.01291297 0.03776419 0.04678887 0.0170784 ], 本步得分 = 1.0, 结束指示 = False, 其他信息 = {}
4: 动作 = 1
4: 观测 = [ 0.01366825  0.23218499  0.04713044 -0.26048273], 本步得分 = 1.0, 结束指示 = False, 其他信息 = {}
5: 动作 = 1
5: 观测 = [ 0.01831195  0.42660357  0.04192079 -0.53793554], 本步得分 = 1.0, 结束指示 = False, 其他信息 = {}
6: 动作 = 1
6: 观测 = [ 0.02684402  0.62111185  0.03116208 -0.81712053], 本步得分 = 1.0, 结束指示 = False, 其他信息 = {}
7: 动作 = 1
7: 观测 = [ 0.03926626  0.81579365  0.01481966 -1.09984129], 本步得分 = 1.0, 结束指示 = False, 其他信息 = {}
8:

玩多局

In [2]:
import gym
env = gym.make('CartPole-v0')
n_episode = 20
for i_episode in range(n_episode):
    observation = env.reset()
    episode_reward = 0
    while True:
        # env.render()
        action = env.action_space.sample() # 随机选
        observation, reward, done, _ = env.step(action)
        episode_reward += reward
        state = observation
        if done:
            break
    print ('第{}局得分 = {}'.format(i_episode, episode_reward))
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
第0局得分 = 14.0
第1局得分 = 19.0
第2局得分 = 14.0
第3局得分 = 12.0
第4局得分 = 16.0
第5局得分 = 25.0
第6局得分 = 13.0
第7局得分 = 41.0
第8局得分 = 22.0
第9局得分 = 13.0
第10局得分 = 19.0
第11局得分 = 20.0
第12局得分 = 16.0
第13局得分 = 21.0
第14局得分 = 26.0
第15局得分 = 12.0
第16局得分 = 12.0
第17局得分 = 12.0
第18局得分 = 19.0
第19局得分 = 58.0


Cart Pole Environment

In [3]:
import gym
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


搭建 DQN

In [4]:
import torch.nn as nn
model = nn.Sequential(
        nn.Linear(env.observation_space.shape[0], 128),
        nn.ReLU(),
        nn.Linear(128, 128),
        nn.ReLU(),
        nn.Linear(128, env.action_space.n)
        )

In [5]:
import random
def act(model, state, epsilon):
    if random.random() > epsilon: # 选最大的
        state = torch.FloatTensor(state).unsqueeze(0)
        q_value = model.forward(state)
        action = q_value.max(1)[1].item()
    else: # 随便选
        action = random.randrange(env.action_space.n)
    return action

训练

In [6]:
# epsilon值不断下降
import math
def calc_epsilon(t, epsilon_start=1.0,
        epsilon_final=0.01, epsilon_decay=500):
    epsilon = epsilon_final + (epsilon_start - epsilon_final) \
            * math.exp(-1. * t / epsilon_decay)
    return epsilon

In [7]:
# 最近历史缓存
import numpy as np
from collections import deque

batch_size = 32

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        state = np.expand_dims(state, 0)
        next_state = np.expand_dims(next_state, 0)
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        state, action, reward, next_state, done = zip( \
                *random.sample(self.buffer, batch_size))
        concat_state = np.concatenate(state)
        concat_next_state = np.concatenate(next_state)
        return concat_state, action, reward, concat_next_state, done
    
    def __len__(self):
        return len(self.buffer)

replay_buffer = ReplayBuffer(1000)

In [8]:
import torch.optim
optimizer = torch.optim.Adam(model.parameters())

gamma = 0.99

episode_rewards = [] # 各局得分,用来判断训练是否完成
t = 0 # 训练步数,用于计算epsilon

while True:
    
    # 开始新的一局
    state = env.reset()
    episode_reward = 0

    while True:
        epsilon = calc_epsilon(t)
        action = act(model, state, epsilon)
        next_state, reward, done, _ = env.step(action)
        replay_buffer.push(state, action, reward, next_state, done)

        state = next_state
        episode_reward += reward

        if len(replay_buffer) > batch_size:

            # 计算时间差分误差
            sample_state, sample_action, sample_reward, sample_next_state, \
                    sample_done = replay_buffer.sample(batch_size)

            sample_state = torch.tensor(sample_state, dtype=torch.float32)
            sample_action = torch.tensor(sample_action, dtype=torch.int64)
            sample_reward = torch.tensor(sample_reward, dtype=torch.float32)
            sample_next_state = torch.tensor(sample_next_state,
                    dtype=torch.float32)
            sample_done = torch.tensor(sample_done, dtype=torch.float32)
            
            next_qs = model(sample_next_state)
            next_q, _ = next_qs.max(1)
            expected_q = sample_reward + gamma * next_q * (1 - sample_done)
            
            qs = model(sample_state)
            q = qs.gather(1, sample_action.unsqueeze(1)).squeeze(1)
            
            td_error = expected_q - q
            
            # 计算 MSE 损失
            loss = td_error.pow(2).mean() 
            
            # 根据损失改进网络
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            t += 1
            
        if done: # 本局结束
            i_episode = len(episode_rewards)
            print ('第{}局收益 = {}'.format(i_episode, episode_reward))
            episode_rewards.append(episode_reward)
            break
            
    if len(episode_rewards) > 20 and np.mean(episode_rewards[-20:]) > 195:
        break # 训练结束

第0局收益 = 29.0
第1局收益 = 21.0
第2局收益 = 21.0
第3局收益 = 15.0
第4局收益 = 32.0
第5局收益 = 15.0
第6局收益 = 14.0
第7局收益 = 42.0
第8局收益 = 12.0
第9局收益 = 15.0
第10局收益 = 10.0
第11局收益 = 44.0
第12局收益 = 20.0
第13局收益 = 24.0
第14局收益 = 16.0
第15局收益 = 25.0
第16局收益 = 10.0
第17局收益 = 15.0
第18局收益 = 17.0
第19局收益 = 53.0
第20局收益 = 33.0
第21局收益 = 20.0
第22局收益 = 31.0
第23局收益 = 22.0
第24局收益 = 13.0
第25局收益 = 27.0
第26局收益 = 48.0
第27局收益 = 23.0
第28局收益 = 48.0
第29局收益 = 23.0
第30局收益 = 32.0
第31局收益 = 24.0
第32局收益 = 65.0
第33局收益 = 29.0
第34局收益 = 25.0
第35局收益 = 63.0
第36局收益 = 32.0
第37局收益 = 65.0
第38局收益 = 35.0
第39局收益 = 63.0
第40局收益 = 54.0
第41局收益 = 101.0
第42局收益 = 75.0
第43局收益 = 139.0
第44局收益 = 70.0
第45局收益 = 79.0
第46局收益 = 129.0
第47局收益 = 122.0
第48局收益 = 95.0
第49局收益 = 140.0
第50局收益 = 140.0
第51局收益 = 127.0
第52局收益 = 171.0
第53局收益 = 200.0
第54局收益 = 128.0
第55局收益 = 200.0
第56局收益 = 200.0
第57局收益 = 200.0
第58局收益 = 200.0
第59局收益 = 184.0
第60局收益 = 200.0
第61局收益 = 200.0
第62局收益 = 200.0
第63局收益 = 200.0
第64局收益 = 199.0
第65局收益 = 200.0
第66局收益 = 200.0
第67局收益 = 200.0
第68局收益 = 200.0
第69局收益 = 200.0
第70局收

使用
（固定 $\epsilon$ 的值为0）

In [9]:
n_episode = 20
for i_episode in range(n_episode):
    observation = env.reset()
    episode_reward = 0
    while True:
        # env.render()
        action = act(model, observation, 0)
        observation, reward, done, _ = env.step(action)
        episode_reward += reward
        state = observation
        if done:
            break
    print ('第{}局得分 = {}'.format(i_episode, episode_reward))

第0局得分 = 200.0
第1局得分 = 200.0
第2局得分 = 200.0
第3局得分 = 200.0
第4局得分 = 200.0
第5局得分 = 200.0
第6局得分 = 200.0
第7局得分 = 200.0
第8局得分 = 200.0
第9局得分 = 200.0
第10局得分 = 200.0
第11局得分 = 200.0
第12局得分 = 200.0
第13局得分 = 200.0
第14局得分 = 200.0
第15局得分 = 200.0
第16局得分 = 200.0
第17局得分 = 200.0
第18局得分 = 200.0
第19局得分 = 200.0
